<a href="https://colab.research.google.com/github/flohmannjr/PJI410/blob/main/SRAG_2021_a_2023S1_openDataSUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SRAG 2021 a 2023 - Banco de Dados de Síndrome Respiratória Aguda Grave

Dados atualizados até 22 de Fevereiro de 2023

https://opendatasus.saude.gov.br/dataset/srag-2021-a-2023

# Importações

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Configurações

In [2]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100

plt.style.use('seaborn-darkgrid')

SEED     = 2001088 + 2001247 + 2003061 + 2008193 + 2008620 + 2008880
COLORMAP = 'summer_r'

# Dados

In [3]:
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.001
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.002
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.003
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.004

!7z e /content/openDataSUS.7z.001

--2023-02-24 12:11:06--  https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.001
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25165824 (24M) [application/octet-stream]
Saving to: ‘openDataSUS.7z.001’

openDataSUS.7z.001  100%[===================>]  24.00M  87.3MB/s    in 0.3s    

2023-02-24 12:11:08 (87.3 MB/s) - ‘openDataSUS.7z.001’ saved [25165824/25165824]

--2023-02-24 12:11:08--  https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.002
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [4]:
CAMPOS = {'DT_NOTIFIC': 'string', 'SEM_NOT': 'Int32', 'DT_SIN_PRI': 'string', 'SEM_PRI': 'Int32', 'SG_UF_NOT': 'string', 'ID_REGIONA': 'string', 'CO_REGIONA': 'Int32',
          'ID_MUNICIP': 'string', 'CO_MUN_NOT': 'Int32', 'ID_UNIDADE': 'string', 'CO_UNI_NOT': 'Int32', 'CS_SEXO': 'string', 'DT_NASC': 'string', 'NU_IDADE_N': 'Int32',
          'TP_IDADE': 'Int32', 'COD_IDADE': 'string', 'CS_GESTANT': 'Int32', 'CS_RACA': 'Int32', 'CS_ESCOL_N': 'Int32', 'ID_PAIS': 'string', 'CO_PAIS': 'Int32',
          'SG_UF': 'string', 'ID_RG_RESI': 'string', 'CO_RG_RESI': 'Int32', 'ID_MN_RESI': 'string', 'CO_MUN_RES': 'Int32', 'CS_ZONA': 'Int32', 'SURTO_SG': 'string',
          'NOSOCOMIAL': 'Int32', 'AVE_SUINO': 'Int32', 'FEBRE': 'Int32', 'TOSSE': 'Int32', 'GARGANTA': 'Int32', 'DISPNEIA': 'Int32', 'DESC_RESP': 'Int32', 'SATURACAO': 'Int32',
          'DIARREIA': 'Int32', 'VOMITO': 'Int32', 'OUTRO_SIN': 'Int32', 'OUTRO_DES': 'string', 'PUERPERA': 'Int32', 'FATOR_RISC': 'Int32', 'CARDIOPATI': 'Int32',
          'HEMATOLOGI': 'Int32', 'SIND_DOWN': 'Int32', 'HEPATICA': 'Int32', 'ASMA': 'Int32', 'DIABETES': 'Int32', 'NEUROLOGIC': 'Int32', 'PNEUMOPATI': 'Int32',
          'IMUNODEPRE': 'Int32', 'RENAL': 'Int32', 'OBESIDADE': 'Int32', 'OBES_IMC': 'string', 'OUT_MORBI': 'Int32', 'MORB_DESC': 'string', 'VACINA': 'Int32',
          'DT_UT_DOSE': 'string', 'MAE_VAC': 'Int32', 'DT_VAC_MAE': 'string', 'M_AMAMENTA': 'Int32', 'DT_DOSEUNI': 'string', 'DT_1_DOSE': 'string', 'DT_2_DOSE': 'string',
          'ANTIVIRAL': 'Int32', 'TP_ANTIVIR': 'Int32', 'OUT_ANTIV': 'string', 'DT_ANTIVIR': 'string', 'HOSPITAL': 'Int32', 'DT_INTERNA': 'string', 'SG_UF_INTE': 'string',
          'ID_RG_INTE': 'string', 'CO_RG_INTE': 'Int32', 'ID_MN_INTE': 'string', 'CO_MU_INTE': 'Int32', 'UTI': 'Int32', 'DT_ENTUTI': 'string', 'DT_SAIDUTI': 'string',
          'SUPORT_VEN': 'Int32', 'RAIOX_RES': 'Int32', 'RAIOX_OUT': 'string', 'DT_RAIOX': 'string', 'AMOSTRA': 'Int32', 'DT_COLETA': 'string', 'TP_AMOSTRA': 'Int32',
          'OUT_AMOST': 'string', 'PCR_RESUL': 'Int32', 'DT_PCR': 'string', 'POS_PCRFLU': 'Int32', 'TP_FLU_PCR': 'Int32', 'PCR_FLUASU': 'Int32', 'FLUASU_OUT': 'string',
          'PCR_FLUBLI': 'Int32', 'FLUBLI_OUT': 'string', 'POS_PCROUT': 'Int32', 'PCR_VSR': 'Int32', 'PCR_PARA1': 'Int32', 'PCR_PARA2': 'Int32', 'PCR_PARA3': 'Int32',
          'PCR_PARA4': 'Int32', 'PCR_ADENO': 'Int32', 'PCR_METAP': 'Int32', 'PCR_BOCA': 'Int32', 'PCR_RINO': 'Int32', 'PCR_OUTRO': 'Int32', 'DS_PCR_OUT': 'string',
          'CLASSI_FIN': 'Int32', 'CLASSI_OUT': 'string', 'CRITERIO': 'Int32', 'EVOLUCAO': 'Int32', 'DT_EVOLUCA': 'string', 'DT_ENCERRA': 'string', 'DT_DIGITA': 'string',
          'HISTO_VGM': 'string', 'PAIS_VGM': 'string', 'CO_PS_VGM': 'string', 'LO_PS_VGM': 'string', 'DT_VGM': 'string', 'DT_RT_VGM': 'string', 'PCR_SARS2': 'Int32',
          'PAC_COCBO': 'string', 'PAC_DSCBO': 'string', 'OUT_ANIM': 'string', 'DOR_ABD': 'Int32', 'FADIGA': 'Int32', 'PERD_OLFT': 'Int32', 'PERD_PALA': 'Int32',
          'TOMO_RES': 'Int32', 'TOMO_OUT': 'string', 'DT_TOMO': 'string', 'TP_TES_AN': 'Int32', 'DT_RES_AN': 'string', 'RES_AN': 'Int32', 'POS_AN_FLU': 'Int32',
          'TP_FLU_AN': 'Int32', 'POS_AN_OUT': 'Int32', 'AN_SARS2': 'Int32', 'AN_VSR': 'Int32', 'AN_PARA1': 'Int32', 'AN_PARA2': 'Int32', 'AN_PARA3': 'Int32',
          'AN_ADENO': 'Int32', 'AN_OUTRO': 'Int32', 'DS_AN_OUT': 'string', 'TP_AM_SOR': 'Int32', 'SOR_OUT': 'string', 'DT_CO_SOR': 'string', 'TP_SOR': 'Int32',
          'OUT_SOR': 'string', 'DT_RES': 'string', 'RES_IGG': 'Int32', 'RES_IGM': 'Int32', 'RES_IGA': 'Int32', 'ESTRANG': 'Int32', 'VACINA_COV': 'Int32',
          'DOSE_1_COV': 'string', 'DOSE_2_COV': 'string', 'DOSE_REF': 'string', 'FAB_COV_1': 'string', 'FAB_COV_2': 'string', 'FAB_COVREF': 'string', 'LAB_PR_COV': 'string',
          'LOTE_1_COV': 'string', 'LOTE_2_COV': 'string', 'LOTE_REF': 'string', 'FNT_IN_COV': 'Int32', 'DOSE_2REF': 'string', 'FAB_COVRF2': 'string', 'LOTE_REF2': 'string',
          'TRAT_COV': 'Int32', 'TIPO_TRAT': 'Int32', 'OUT_TRAT': 'string', 'DT_TRT_COV': 'string'}

# AUSENTES = df_23.columns[~df_23.columns.isin(df_21.columns)]
AUSENTES = ['DOSE_2REF', 'FAB_COVRF2', 'LOTE_REF2', 'TRAT_COV', 'TIPO_TRAT', 'OUT_TRAT', 'DT_TRT_COV']

DATAS = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_UT_DOSE', 'DT_VAC_MAE', 'DT_DOSEUNI', 'DT_1_DOSE', 'DT_2_DOSE', 'DT_ANTIVIR', 'DT_INTERNA',
         'DT_ENTUTI', 'DT_SAIDUTI', 'DT_RAIOX', 'DT_COLETA', 'DT_PCR', 'DT_EVOLUCA', 'DT_ENCERRA', 'DT_DIGITA', 'DT_VGM', 'DT_RT_VGM', 'DT_TOMO',
         'DT_RES_AN', 'DT_CO_SOR', 'DT_RES', 'DOSE_1_COV', 'DOSE_2_COV', 'DOSE_REF', 'DOSE_2REF', 'DT_TRT_COV']

In [5]:
df = pd.read_csv('INFLUD21-20-02-2023.csv', sep=';', dtype=dict(list(CAMPOS.items())[:-7]))

In [6]:
df[AUSENTES] = [None] * len(AUSENTES)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [7]:
df = pd.concat([df, pd.read_csv('INFLUD22-13-02-2023.csv', sep=';', dtype=CAMPOS)])
df = pd.concat([df, pd.read_csv('INFLUD23-13-02-2023.csv', sep=';', dtype=CAMPOS)])

In [8]:
for campo in DATAS:
    df[campo] = pd.to_datetime(df[campo], format='%d/%m/%Y', errors='coerce')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310360 entries, 0 to 22642
Columns: 173 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: Int32(99), datetime64[ns](29), string(45)
memory usage: 2.4 GB


In [10]:
# df.info(max_cols=180, show_counts=True)

## Pré-processamento

In [11]:
# with pd.option_context('display.max_columns', None): 
#     display(df[DATAS[:-1]].describe(datetime_is_numeric=True))

In [12]:
# with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
#     display(df[df['DT_NASC'].isna()])

In [13]:
# with pd.option_context('display.max_rows', None): 
#     display(df['DT_NASC'].dt.year.value_counts().sort_index())

In [ ]:
# INEXISTENTES = ['TEM_CPF', 'NU_CPF', 'NU_CNS', 'NM_PACIENT', 'TP_POV_CT', 'NM_MAE_PAC', 'NM_BAIRRO', 'NM_LOGRADO', 'NU_NUMERO', 'NM_COMPLEM',
#                 'NU_DDD_TEL', 'NU_TELEFON', 'FAB_COV1', 'FAB_COV2', 'FAB_COVRF', 'ID_UN_INTE', 'CO_UN_INTE', 'REQUI_GAL', 'LAB_AN', 'CO_LAB_AN',
#                 'LAB_PCR', 'CO_LAB_PCR', 'NU_DO', 'OBSERVA', 'NOME_PROF', 'REG_PROF', 'CS_ETINIA', 'POV_CT', 'NU_CEP']

In [28]:
CAMPOS = ['NU_IDADE_N', 'TP_IDADE', 'ID_UNIDADE', 'CO_UNI_NOT', 'OUT_ANIM', 'OUTRO_DES', 'OBES_IMC', 'MORB_DESC', 'FAB_COVRF2',
          'LOTE_1_COV', 'LOTE_2_COV', 'LOTE_REF', 'LOTE_REF2', 'OUT_ANTIV', 'OUT_TRAT', 'RAIOX_OUT', 'TOMO_OUT', 'OUT_AMOST',
          'DS_AN_OUT', 'FLUASU_OUT', 'FLUBLI_OUT', 'DS_PCR_OUT', 'SOR_OUT', 'OUT_SOR', 'SOR_OUT', 'CLASSI_OUT',
          'ID_MUNICIP', 'CO_MUN_NOT', 'ID_REGIONA', 'CO_REGIONA', 'PAC_COCBO', 'PAC_DSCBO', 'ID_RG_RESI', 'CO_RG_RESI',
          'ID_MN_RESI', 'CO_MUN_RES', 'ID_RG_INTE', 'CO_RG_INTE', 'ID_MN_INTE', 'CO_MU_INTE']

In [29]:
df[CAMPOS].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310360 entries, 0 to 22642
Data columns (total 40 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   NU_IDADE_N  2310360 non-null  Int32 
 1   TP_IDADE    2310360 non-null  Int32 
 2   ID_UNIDADE  2310360 non-null  string
 3   CO_UNI_NOT  2310360 non-null  Int32 
 4   OUT_ANIM    2019 non-null     string
 5   OUTRO_DES   694988 non-null   string
 6   OBES_IMC    23894 non-null    string
 7   MORB_DESC   552988 non-null   string
 8   FAB_COVRF2  33122 non-null    string
 9   LOTE_1_COV  590153 non-null   string
 10  LOTE_2_COV  493157 non-null   string
 11  LOTE_REF    192595 non-null   string
 12  LOTE_REF2   33170 non-null    string
 13  OUT_ANTIV   6815 non-null     string
 14  OUT_TRAT    709 non-null      string
 15  RAIOX_OUT   78938 non-null    string
 16  TOMO_OUT    62783 non-null    string
 17  OUT_AMOST   186145 non-null   string
 18  DS_AN_OUT   1299 non-null     string
 19  FL

In [40]:
CAMPOS = ['CS_GESTANT', 'SG_UF_INTE', 'RES_IGG', 'RES_IGM', 'RES_IGA']

In [41]:
df[CAMPOS].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310360 entries, 0 to 22642
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   CS_GESTANT  2310360 non-null  Int32 
 1   SG_UF_INTE  2239369 non-null  string
 2   RES_IGG     256592 non-null   Int32 
 3   RES_IGM     259464 non-null   Int32 
 4   RES_IGA     202793 non-null   Int32 
dtypes: Int32(4), string(1)
memory usage: 79.3 MB


In [42]:
with pd.option_context('display.max_rows', None): 
    for campo in CAMPOS:
        display(df[campo].value_counts(dropna=False).sort_index())
        print()

0          378
1         2511
2         5809
3        14226
4         1216
5       694357
6      1498864
9        92999
NaN          0
Name: CS_GESTANT, dtype: Int64

AC       6190
AL      25035
AM      30790
AP       5336
BA      81904
CE      92094
DF      49562
ES      17048
GO      75496
MA      23973
MG     273612
MS      43029
MT      29289
PA      45021
PB      32590
PE      74124
PI      20526
PR     191573
RJ     176191
RN      20755
RO      16882
RR       4149
RS     128840
SC      90859
SE      23334
SP     645394
TO      15773
NaN     70991
Name: SG_UF_INTE, dtype: Int64

1        66689
2        42115
3          230
4       142326
5          401
9         4831
NaN    2053768
Name: RES_IGG, dtype: Int64

1        72106
2        41409
3          281
4       140601
5          429
9         4638
NaN    2050896
Name: RES_IGM, dtype: Int64

1         6174
2         4952
3          409
4       182458
5          586
9         8214
NaN    2107567
Name: RES_IGA, dtype: Int64

### Correção

In [20]:
# Campo CLASSI_FIN com valores NaN para 4-SRAG não especificado
df['CLASSI_FIN'].fillna(4, inplace=True)

# Campo CS_SEXO para numérico, como dicionário de dados.
df['CS_SEXO'].replace({'M': 1, 'F': 2, 'I': 9}, inplace=True)
df['CS_SEXO'] = df['CS_SEXO'].astype('Int32')

# Campo ESTRANG com valores NaN para 2-Não
df['ESTRANG'].fillna(2, inplace=True)

# Campos com valores NaN para 9-Ignorado
NAN9 = ['AMOSTRA', 'ANTIVIRAL', 'ASMA', 'AVE_SUINO', 'CARDIOPATI', 'CRITERIO', 'CS_ESCOL_N', 'CS_RACA', 'CS_ZONA', 'DESC_RESP', 'DIABETES',
        'DIARREIA', 'DISPNEIA', 'DOR_ABD', 'EVOLUCAO', 'FADIGA', 'FATOR_RISC', 'FEBRE', 'FNT_IN_COV', 'GARGANTA', 'HEMATOLOGI', 'HEPATICA',
        'HOSPITAL', 'IMUNODEPRE', 'MAE_VAC', 'M_AMAMENTA', 'NEUROLOGIC', 'NOSOCOMIAL', 'OBESIDADE', 'OUTRO_SIN', 'OUT_MORBI', 'PCR_FLUASU',
        'PCR_FLUBLI', 'PCR_RESUL', 'PERD_OLFT', 'PERD_PALA', 'PNEUMOPATI', 'POS_AN_FLU', 'POS_AN_OUT', 'POS_PCRFLU', 'POS_PCROUT', 'PUERPERA',
        'RAIOX_RES', 'RENAL', 'RES_AN', 'SATURACAO', 'SIND_DOWN', 'SUPORT_VEN', 'TIPO_TRAT', 'TOMO_RES', 'TOSSE', 'TP_AMOSTRA', 'TP_AM_SOR',
        'TP_ANTIVIR', 'TP_FLU_AN', 'TP_FLU_PCR', 'TP_SOR', 'TP_TES_AN', 'TRAT_COV', 'UTI', 'VACINA', 'VACINA_COV', 'VOMITO']

for campo in NAN9:
    df[campo].fillna(9, inplace=True)

### Contagem

In [38]:
CAMPOS = ['DT_NOTIFIC', 'SEM_NOT', 'DT_SIN_PRI', 'SEM_PRI', 'SG_UF_NOT', 'ID_PAIS', 'CO_PAIS']

In [39]:
df[CAMPOS].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310360 entries, 0 to 22642
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   DT_NOTIFIC  2310360 non-null  datetime64[ns]
 1   SEM_NOT     2310360 non-null  Int32         
 2   DT_SIN_PRI  2310360 non-null  datetime64[ns]
 3   SEM_PRI     2310360 non-null  Int32         
 4   SG_UF_NOT   2310360 non-null  string        
 5   ID_PAIS     2310360 non-null  string        
 6   CO_PAIS     2310360 non-null  Int32         
dtypes: Int32(3), datetime64[ns](2), string(2)
memory usage: 121.2 MB


In [32]:
df[DATAS[:-1]].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310360 entries, 0 to 22642
Data columns (total 28 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   DT_NOTIFIC  2310360 non-null  datetime64[ns]
 1   DT_SIN_PRI  2310360 non-null  datetime64[ns]
 2   DT_NASC     2308970 non-null  datetime64[ns]
 3   DT_UT_DOSE  92107 non-null    datetime64[ns]
 4   DT_VAC_MAE  1186 non-null     datetime64[ns]
 5   DT_DOSEUNI  236 non-null      datetime64[ns]
 6   DT_1_DOSE   239 non-null      datetime64[ns]
 7   DT_2_DOSE   189 non-null      datetime64[ns]
 8   DT_ANTIVIR  51194 non-null    datetime64[ns]
 9   DT_INTERNA  2157993 non-null  datetime64[ns]
 10  DT_ENTUTI   637296 non-null   datetime64[ns]
 11  DT_SAIDUTI  383907 non-null   datetime64[ns]
 12  DT_RAIOX    468513 non-null   datetime64[ns]
 13  DT_COLETA   2097590 non-null  datetime64[ns]
 14  DT_PCR      1481952 non-null  datetime64[ns]
 15  DT_EVOLUCA  1940601 non-null  date

In [21]:
# Quantidade de registros com sexo diferente de 1-Masculino ou 2-Feminino.
(~df['CS_SEXO'].isin([1, 2])).sum()

376

In [22]:
# Quantidade de registros sem data de nascimento.
df['DT_NASC'].isna().sum()

1390

In [ ]:
# # Quantidade de registros com ano de nascimento inferior a 1923, por ano.
# with pd.option_context('display.max_rows', None): 
#     display(df['DT_NASC'][df['DT_NASC'].dt.year < 1923].dt.year.value_counts().sort_index())

In [24]:
# Quantidade total de registros com ano de nascimento inferior a 1923.
(df['DT_NASC'].dt.year < 1923).sum()

6137

In [25]:
# Quantidade de registros sem unidade federativa.
df['SG_UF'].isna().sum()

398

### Limpeza

In [26]:
# Remover colunas vazias.
df.dropna(how='all', axis=1, inplace=True)

# Remover registros com sexo diferente de 1-Masculino ou 2-Feminino.
df.drop(df[~df['CS_SEXO'].isin([1, 2])].index, inplace=True)

# Remover registros sem data de nascimento ou unidade federativa.
df.dropna(subset=['DT_NASC', 'SG_UF'], inplace=True)

# Remover registros com ano de nascimento inferior a 1923.
df.drop(df[df['DT_NASC'].dt.year < 1923].index, inplace=True)